In [2]:
import pandas as pd
import numpy as np

                                    MASUKINNN NAMA FILE EXCEL DATA KARYAWAN LENGKAP                                    

In [3]:
data_mentah_karyawan = pd.read_excel('absensi 2024.xlsx') # MASUKIN NAMA FILE DATA KARYAWAN LENGKAP DISINI
data_mentah_karyawan.drop(columns='No')
data_karyawan = data_mentah_karyawan.dropna(axis=0, how='all')
data_karyawan['NEW MANUAL'] = data_karyawan['NEW MANUAL'].astype(str).str.extract(r'(\d+)', expand=False).astype('Int64')
data_karyawan = data_karyawan.sort_values(by='NEW MANUAL')
data_karyawan = data_karyawan.reset_index(drop=True)
data_karyawan

,No,NAMA KARYAWAN,NEW MANUAL,NEW SISTEM,UNIT KERJA,KHUSUS
0,75,"DARSONO, S KOM",12,1207080061 - DARSONO,KA. LAB.KOMPUTER,30000
1,74,"ABDUL ROKIB, S.KOM",13,1203060045 - ABDUL ROKIB,KOOR. PEL. PUSTIKOM,30000
2,48,"FIRRA ANGRAINI, S.Ak.",16,1218080240 - FIRRA ANGRAINI,STAF BIRO KEUANGAN,30000
3,42,"FIRDAUS SYAMSU, SIP.,MKM.",18,1212030108 - FIRDAUS SYAMSU,STAF BIRO SDM,30000
4,32,SADI,19,1211010081 - SADI,OPERATOR FOTOCOPY,30000
...,...,...,...,...,...,...
181,169,DEWI AISAH,<NA>,NaN,MHS Magang,0
182,170,ROSELINA WIDIA ARTANTI,<NA>,NaN,MHS Magang,0
183,172,RIYANTI,<NA>,1215120199 - RIYANTI,CS,0
184,173,"YUYUN KURNIASIH, S.Kp., SAP., M.Kep.",<NA>,NaN,KA. UPT,0


                                     MASUKINNN NAMA FILE EXCEL DATA FINGERPRINT                                           

In [4]:
data_mentah_finger = pd.read_excel("09-17 jan.xls") # MASUKIN NAMA FILE FINGER DISINI #
data_mentah_finger['tanggal'] = pd.to_datetime(data_mentah_finger ['Unnamed: 3'], format='%d/%m/%Y %H:%M')
data_mentah_finger['tanggal'] = data_mentah_finger['tanggal'].dt.date

data_finger = data_mentah_finger.drop(columns='Unnamed: 0')
data_finger['tanggals'] = pd.to_datetime(data_finger['Unnamed: 3'], format='%d/%m/%Y %H:%M')
data_finger['jam'] = data_finger['tanggals'].dt.strftime('%H:%M')

data_finger = data_finger.drop(columns=['tanggals'])
data_finger.drop(columns='Unnamed: 7', inplace=True)
data_finger.drop(columns='Unnamed: 6', inplace=True)
data_finger.rename(columns={'Unnamed: 1': 'NAMA KARYAWAN'}, inplace=True)
data_finger.rename(columns={'Unnamed: 2': 'NEW MANUAL'}, inplace=True)
data_finger.head(10)

,NAMA KARYAWAN,NEW MANUAL,Unnamed: 3,Unnamed: 4,Unnamed: 5,tanggal,jam
0,Darsono,12,09/01/2024 9:02,C/Masuk,1,2024-01-09,09:02
1,Darsono,12,10/01/2024 9:16,C/Masuk,1,2024-01-10,09:16
2,Darsono,12,10/01/2024 16:24,C/Masuk,1,2024-01-10,16:24
3,Darsono,12,12/01/2024 10:54,C/Masuk,1,2024-01-12,10:54
4,Darsono,12,15/01/2024 9:22,C/Masuk,1,2024-01-15,09:22
5,Darsono,12,2024-02-01 09:48:00,C/Masuk,1,2024-02-01,09:48
6,Abdul rokib,13,09/01/2024 16:40,C/Keluar,1,2024-01-09,16:40
7,Abdul rokib,13,10/01/2024 9:11,C/Masuk,1,2024-01-10,09:11
8,Abdul rokib,13,10/01/2024 18:36,C/Keluar,1,2024-01-10,18:36
9,Abdul rokib,13,11/01/2024 8:42,C/Masuk,1,2024-01-11,08:42


                                 MASUKINNN TANGGAL FINGER MAU DARI TANGGAL BERAPA SAMPAI TANGGAL BERAPA                   

In [5]:
                        #################   MASUKAN TANGGAL FINGER DISINI  #################

tanggal_finger = pd.date_range(start='2024-01-09', end='2024-02-01', freq='D')
data_finger_hampir_fix = pd.DataFrame(columns=['NEW MANUAL', 'NAMA KARYAWAN'] + tanggal_finger.strftime('%d/%m/%Y').tolist())

start_date_finger = pd.to_datetime('2024-01-09')
end_date_finger = pd.to_datetime('2024-02-01')

data_finger['tanggal'] = pd.to_datetime(data_finger['tanggal'])

for name, group in data_finger.groupby('NEW MANUAL'):
    new_row = {'NEW MANUAL': name, 'NAMA KARYAWAN': group['NAMA KARYAWAN'].iloc[0]}
    
    current_date = start_date_finger
    while current_date <= end_date_finger:
        formatted_date = current_date.strftime('%d/%m/%Y')
        new_row[formatted_date] = "" if ((group['tanggal'].dt.day == current_date.day) & (group['Unnamed: 5'] == 1)).any() else ''
        current_date += pd.DateOffset(days=1)
    
    data_finger_hampir_fix = pd.concat([data_finger_hampir_fix, pd.DataFrame([new_row])], ignore_index=True)

data_finger_hampir_fix = data_finger_hampir_fix.sort_values(by='NEW MANUAL')

for index, row in data_finger.iterrows():
    tanggal = row['tanggal'].strftime('%d/%m/%Y')
    status = row['Unnamed: 4']
    jam = row['jam']

    if status == 'C/Masuk':
        if pd.isna(data_finger_hampir_fix.loc[data_finger_hampir_fix['NEW MANUAL'] == row['NEW MANUAL'], tanggal].iloc[0]):
            data_finger_hampir_fix.loc[data_finger_hampir_fix['NEW MANUAL'] == row['NEW MANUAL'], tanggal] = f'Masuk ({jam})'
        else:
            existing_data = data_finger_hampir_fix.loc[data_finger_hampir_fix['NEW MANUAL'] == row['NEW MANUAL'], tanggal].iloc[0]
            data_finger_hampir_fix.loc[data_finger_hampir_fix['NEW MANUAL'] == row['NEW MANUAL'], tanggal] = f'{existing_data} masuk ({jam})'
    elif status == 'C/Keluar':
        if pd.isna(data_finger_hampir_fix.loc[data_finger_hampir_fix['NEW MANUAL'] == row['NEW MANUAL'], tanggal].iloc[0]):
            data_finger_hampir_fix.loc[data_finger_hampir_fix['NEW MANUAL'] == row['NEW MANUAL'], tanggal] = f'Keluar ({jam})'
        else:
            existing_data = data_finger_hampir_fix.loc[data_finger_hampir_fix['NEW MANUAL'] == row['NEW MANUAL'], tanggal].iloc[0]
            data_finger_hampir_fix.loc[data_finger_hampir_fix['NEW MANUAL'] == row['NEW MANUAL'], tanggal] = f'{existing_data} Keluar ({jam})'

data_finger_fix = data_finger_hampir_fix.fillna('Tidak Masuk')
data_finger_fix = data_finger_fix.applymap(lambda x: 'Tidak Masuk' if x == '' else x)
data_finger_fix

C:\Users\fpamu\AppData\Local\Temp\ipykernel_9780\2415883103.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_finger_fix = data_finger_fix.applymap(lambda x: 'Tidak Masuk' if x == '' else x)


,NEW MANUAL,NAMA KARYAWAN,09/01/2024,10/01/2024,11/01/2024,12/01/2024,13/01/2024,14/01/2024,15/01/2024,16/01/2024,...,23/01/2024,24/01/2024,25/01/2024,26/01/2024,27/01/2024,28/01/2024,29/01/2024,30/01/2024,31/01/2024,01/02/2024
0,12,Darsono,masuk (09:02),masuk (09:16) masuk (16:24),Tidak Masuk,masuk (10:54),Tidak Masuk,Tidak Masuk,masuk (09:22),Tidak Masuk,...,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,masuk (09:48)
1,13,Abdul rokib,Keluar (16:40),masuk (09:11) Keluar (18:36),masuk (08:42),masuk (08:52) masuk (17:24) Keluar (17:32),Tidak Masuk,Tidak Masuk,masuk (08:43) masuk (17:35) Keluar (17:50),masuk (08:19) Keluar (17:47),...,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
2,16,Firra,masuk (07:34) Keluar (17:00),masuk (07:37) Keluar (16:58),masuk (07:34) Keluar (17:00),masuk (07:32) Keluar (17:05),Tidak Masuk,Tidak Masuk,masuk (07:24) Keluar (17:02),masuk (07:18) Keluar (17:02),...,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
3,18,Firdaus,Keluar (18:32),Keluar (17:33),Tidak Masuk,Keluar (17:15),Tidak Masuk,Tidak Masuk,Keluar (17:19),masuk (09:50) Keluar (17:06),...,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
4,19,Sadi,masuk (08:39) Keluar (21:43),masuk (06:19) Keluar (17:15),masuk (07:34) Keluar (19:49),masuk (07:31) Keluar (16:56),Tidak Masuk,Tidak Masuk,masuk (09:32) Keluar (17:45),masuk (07:36) Keluar (21:00),...,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,1235,FERY HOTMAJAYA,masuk (08:34) Keluar (17:07),masuk (08:09) Keluar (17:12),masuk (07:30) Keluar (17:08),masuk (08:28),Tidak Masuk,Tidak Masuk,masuk (07:54) Keluar (17:10),masuk (08:18) Keluar (17:20),...,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
157,1236,Bucika Hasanah Putri,masuk (08:23) Keluar (18:32),masuk (08:19) Keluar (17:33),masuk (08:17) Keluar (17:53),masuk (08:22) Keluar (17:15),Tidak Masuk,Tidak Masuk,masuk (07:43) Keluar (17:05),masuk (08:25) Keluar (17:07),...,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
158,1237,Rahmi Andrita Yuda,masuk (07:41) Keluar (19:09),masuk (07:51) Keluar (18:43),masuk (07:24) Keluar (19:10),masuk (07:40),Tidak Masuk,Tidak Masuk,Keluar (18:47),masuk (08:59),...,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
159,1238,Dwi Prihatiningsih,masuk (07:51) Keluar (17:02),masuk (07:56) Keluar (17:09),masuk (07:49) Keluar (17:06),masuk (07:56) Keluar (16:54),Tidak Masuk,Tidak Masuk,masuk (07:55) Keluar (17:01),masuk (07:57) Keluar (17:04),...,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk


                                                   MERGER DATA FINGER DAN DATA KARYAWAN LENGKAP                       

In [6]:
merged_data = pd.merge(data_karyawan, data_finger_fix, on='NEW MANUAL', how='left', suffixes=('', 'di finger'))
merged_data.drop(columns='NAMA KARYAWANdi finger', inplace=True)
merged_data

,No,NAMA KARYAWAN,NEW MANUAL,NEW SISTEM,UNIT KERJA,KHUSUS,09/01/2024,10/01/2024,11/01/2024,12/01/2024,...,23/01/2024,24/01/2024,25/01/2024,26/01/2024,27/01/2024,28/01/2024,29/01/2024,30/01/2024,31/01/2024,01/02/2024
0,75,"DARSONO, S KOM",12,1207080061 - DARSONO,KA. LAB.KOMPUTER,30000,masuk (09:02),masuk (09:16) masuk (16:24),Tidak Masuk,masuk (10:54),...,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,masuk (09:48)
1,74,"ABDUL ROKIB, S.KOM",13,1203060045 - ABDUL ROKIB,KOOR. PEL. PUSTIKOM,30000,Keluar (16:40),masuk (09:11) Keluar (18:36),masuk (08:42),masuk (08:52) masuk (17:24) Keluar (17:32),...,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
2,48,"FIRRA ANGRAINI, S.Ak.",16,1218080240 - FIRRA ANGRAINI,STAF BIRO KEUANGAN,30000,masuk (07:34) Keluar (17:00),masuk (07:37) Keluar (16:58),masuk (07:34) Keluar (17:00),masuk (07:32) Keluar (17:05),...,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
3,42,"FIRDAUS SYAMSU, SIP.,MKM.",18,1212030108 - FIRDAUS SYAMSU,STAF BIRO SDM,30000,Keluar (18:32),Keluar (17:33),Tidak Masuk,Keluar (17:15),...,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
4,32,SADI,19,1211010081 - SADI,OPERATOR FOTOCOPY,30000,masuk (08:39) Keluar (21:43),masuk (06:19) Keluar (17:15),masuk (07:34) Keluar (19:49),masuk (07:31) Keluar (16:56),...,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,169,DEWI AISAH,<NA>,NaN,MHS Magang,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,170,ROSELINA WIDIA ARTANTI,<NA>,NaN,MHS Magang,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,172,RIYANTI,<NA>,1215120199 - RIYANTI,CS,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,173,"YUYUN KURNIASIH, S.Kp., SAP., M.Kep.",<NA>,NaN,KA. UPT,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


                                      INPUT ABSEN MANUAL SURAT TUGAS DAN MASUKAN TANGGAL LAGI                                                                            

In [20]:
data_merger = merged_data

mapping_dict = dict(zip(data_mentah_karyawan['NAMA KARYAWAN'], data_mentah_karyawan.index))

data_merger['Urutan'] = data_merger['NAMA KARYAWAN'].map(mapping_dict)

data_absen_rapih = data_merger.sort_values(by=['Urutan']).drop('Urutan', axis=1).reset_index(drop=True)

data_karyawan_bersih = data_mentah_karyawan.dropna(axis=0, how='all')
data_karyawan_bersih_juga = data_karyawan_bersih.reset_index(drop=True)
data_karyawan_bersih_banget = data_karyawan_bersih_juga[['NEW MANUAL']]

data_karyawan_bersih_fix = data_karyawan_bersih_banget.reset_index(drop=True)
data_absen_rapih['NEW MANUAL'] = data_karyawan_bersih_fix['NEW MANUAL']
data_absen_rapih['NEW MANUAL'].fillna('absen manual', inplace=True)

###################### MASUKIN TANGGAL DIBAWAH #####################
for index, row in data_absen_rapih.iterrows():
    if 'absen manual' not in row['NEW MANUAL'].lower() and row['09/01/2024':'01/02/2024'].isna().all():
        data_absen_rapih.loc[index, '09/01/2024':'01/02/2024'] = 'Tidak Masuk'

        ######## SETELAH DI RUN INPUT MANUAL DI EXCEL, LALU LANJUT UNTUK MEMBACA FILE EXCEL YANG SUDAH DI INPUT, HAPUS TANDA KURUNG DIBAWAH ########

# data_absen_rapih.to_excel("per bulan.xlsx",index=False) # TUTUP LAGI DENGAN TANDA KURUNG JIKA SUDAH DI RUN



# absen_pertanggal_20 = pd.read_excel('per bulan.xlsx.xlsx') # TUTUP LAGI DENGAN TANDA KURUNG JIKA SUDAH DI RUN



IndentationError: expected an indented block (2293858905.py, line 21)

                                                        MASUKINNN TANGGAL MERAH DI BULAN TERSEBUT                                                 

In [292]:
from datetime import datetime, timedelta

############################################# MASUKAN TANGGAL MERAH  #################################################
tanggal_merah = ["2024-01-09"]
tanggal_columnsk = absen_pertanggal_20.columns[6:]
def hitung_jumlah_ga_masuk(row):
    jumlah_tidak_masuk = 0
    for col in absen_pertanggal_20.columns[6:]: 
        if pd.notna(row[col]) and "Tidak Masuk" in str(row[col]) and pd.to_datetime(col, format="%d/%m/%Y").weekday() < 5 and col not in tanggal_merah:
            jumlah_tidak_masuk += 1

    return jumlah_tidak_masuk
def hitung_sisa_cuti(row):
    jumlah_tidak_masuk = 0
    for col in absen_pertanggal_20.columns[6:-1]:  
        if pd.notna(row[col]) and "Tidak Masuk" in str(row[col]) and pd.to_datetime(col, format="%d/%m/%Y").weekday() < 5 and col not in tanggal_merah:
            jumlah_tidak_masuk += 1

    return 12 - jumlah_tidak_masuk

absen_pertanggal_20["Jumlah Tidak Masuk"] = absen_pertanggal_20.apply(hitung_jumlah_ga_masuk, axis=1)
absen_pertanggal_20["Sisa Cuti"] = absen_pertanggal_20.apply(hitung_sisa_cuti, axis=1)
absen_pertanggal_20['JUMLAH TMK'] = 0

############################################# MASUKAN TANGGAL MERAH  #################################################
tanggal_merah = datetime(2024, 1, 9)

for index, row in absen_pertanggal_20.iterrows():
    for col in absen_pertanggal_20.columns[6:-3]:  
        if pd.notna(row[col]) and isinstance(row[col], str) and 'Tidak Masuk' not in row[col]:
            tanggal = datetime.strptime(col, '%d/%m/%Y')
            if tanggal.weekday() >= 5 or tanggal == tanggal_merah:
                if '(st)' in row[col] or '(pkt)' in row[col]:
                    if row['TMK'] == 35000:
                        absen_pertanggal_20.at[index, 'JUMLAH TMK'] += 70000
                    elif row['TMK'] == 30000:
                       absen_pertanggal_20.at[index, 'JUMLAH TMK'] += 60000
                    else:
                        absen_pertanggal_20.at[index, 'JUMLAH TMK'] += 0
            else:
                if row['TMK'] == 35000:
                    absen_pertanggal_20.at[index, 'JUMLAH TMK'] += 35000
                elif row['TMK'] == 30000:
                    absen_pertanggal_20.at[index, 'JUMLAH TMK'] += 30000
                elif row['TMK'] == 0:
                    absen_pertanggal_20.at[index, 'JUMLAH TMK'] += 0


total_jumlah_tmk = absen_pertanggal_20['JUMLAH TMK'].sum()
total_df = pd.DataFrame({'NAMA KARYAWAN': ['Total'], 'JUMLAH TMK': [total_jumlah_tmk]})
absen_pertanggal_20= pd.concat([absen_pertanggal_20, total_df], ignore_index=True)
###################### JANGAN LUPA KASIH TANDA KURUNG LAGI JIKA SUDAH DI RUN #################

# absen_pertanggal_20.to_excel('21jan-20feb.xlsx', index= False)
# absen_pertanggal_20.to_excel('21feb-20march.xlsx', index= False)
# absen_pertanggal_20.to_excel('21march-20april.xlsx', index= False)
# absen_pertanggal_20.to_excel('21april-20may.xlsx', index= False)
# absen_pertanggal_20.to_excel('21may-20juni.xlsx', index= False)
# absen_pertanggal_20.to_excel('21juni-20juli.xlsx', index= False)
# absen_pertanggal_20.to_excel('21juli-20agus.xlsx', index= False)
# absen_pertanggal_20.to_excel('21agus-20sept.xlsx', index= False)
# absen_pertanggal_20.to_excel('21sept-20okt.xlsx', index= False)
# absen_pertanggal_20.to_excel('21okt-20nov.xlsx', index= False)
# absen_pertanggal_20.to_excel('21nov-20dec.xlsx', index= False)
# absen_pertanggal_20.to_excel('21dec-20jan.xlsx', index= False)



,No,NAMA KARYAWAN,NEW MANUAL,NEW SISTEM,UNIT KERJA,09/01/2024,10/01/2024,11/01/2024,12/01/2024,13/01/2024,14/01/2024,15/01/2024,16/01/2024,17/01/2024,Sisa Cuti,Jumlah TMK
0,1,"SITI JUMHATI,S.ST.,SKM., M.Kes.",Jumhati-1167,1213090125 - SITI JUMHATI,WAKIL REKTOR I,masuk (08:44) Keluar (18:55),masuk (08:54) Keluar (18:55),masuk (08:49) Keluar (18:49),Keluar (19:48),Tidak Masuk,Tidak Masuk,Tidak Masuk,masuk (10:15) Keluar (19:23),masuk (08:04),11,210000
1,2,"Dr. Dra.SUSIANA DEWI RATIH,MM",Susiana-121,1213110132 - SUSIANA DEWI RATIH,WAKIL REKTOR II,masuk (08:48) Keluar (18:49),masuk (09:12) Keluar (17:42),Tidak Masuk,masuk (08:19) masuk (16:03),Tidak Masuk,Tidak Masuk,masuk (08:34) Keluar (17:04),masuk (09:12) Keluar (18:05),masuk (07:07),11,210000
2,3,"Dr. NUR ASNIATI DJAALI, SKM., MKM",Nur asniati-1110,1216080203 - NUR ASNIATI DJAALI,WAKIL REKTOR III,masuk (09:30) Keluar (20:49),masuk (13:27) Keluar (17:42),masuk (08:51) Keluar (16:58),masuk (10:52),Tidak Masuk,Tidak Masuk,masuk (09:17) Keluar (17:21),masuk (10:22) Keluar (14:45),Tidak Masuk,11,210000
3,4,"SUNDARI FATIMAH, STr.Keb.,MTr.Keb.",Sundari fatimah-1108,1219040263 - SUNDARI FATIMAH,SEKRETARIS REKTOR,masuk (09:09) Keluar (20:51),masuk (10:02) masuk (22:42),masuk (09:46) Keluar (20:08),masuk (09:20) Keluar (20:39),masuk (09:24) masuk (13:58),Tidak Masuk,masuk (09:50) Keluar (22:51),masuk (09:52) Keluar (21:32),Tidak Masuk,11,280000
4,5,"ABDUL WAHID,SIP",A wahid-139,1214030149 - ABDUL WAHID,SEKRETARIS REKTOR,masuk (08:17) Keluar (19:11),masuk (08:54) masuk (22:42),masuk (09:12) Keluar (17:25),masuk (09:22) masuk (16:00),Tidak Masuk,Tidak Masuk,masuk (09:37) Keluar (17:19),masuk (09:01) Keluar (18:45),masuk (07:11),12,245000
5,6,"M. MUHIDIN, S.Kom., MM.",Muhidin-117,1210100077 - MOH. MUHIDIN M.,KA. BIRO AKADEMIK,masuk (08:50) Keluar (16:34),masuk (09:01) Keluar (16:30),masuk (07:51) Keluar (16:31),masuk (08:42) Keluar (16:27),Tidak Masuk,Tidak Masuk,masuk (09:54) Keluar (16:46),masuk (09:07) Keluar (16:28),Tidak Masuk,11,210000
6,7,"SIWI PURWANTI, SE",Siwi-116,1200090032 - SIWI PURWANTI,KOOR. PEL. AKADEMIK & PENDIDIKAN,masuk (08:29) Keluar (17:02),masuk (08:32) Keluar (17:04),masuk (08:33) Keluar (17:17),Keluar (17:03),Tidak Masuk,Tidak Masuk,masuk (09:11) Keluar (17:17),masuk (08:32) Keluar (17:05),masuk (07:27),12,245000
7,8,"SUPRIYADI, S.Kom",Supriyadi-119,"1217040211 - SUPRIYADI, S.Kom",KOOR. PANG. DATA PEND. TINGGI (PDPT),masuk (08:34) Keluar (17:02),masuk (08:50) Keluar (17:09),masuk (08:50) Keluar (17:17),masuk (09:19) Keluar (17:04),Tidak Masuk,Tidak Masuk,masuk (09:15) Keluar (17:17) Keluar (17:22),masuk (09:02) Keluar (17:15),masuk (07:33),12,245000
8,9,"RENDY ALBIANSYAH, S.Kom",Rendi albiansyah-173,1219010254 - RENDY ALBIANSYAH,STAF PDPT,masuk (09:24) Keluar (17:02),masuk (08:37) Keluar (17:09),masuk (09:10) Keluar (17:17),masuk (09:49) Keluar (17:08),Tidak Masuk,Tidak Masuk,masuk (09:12) Keluar (17:13),masuk (08:46) Keluar (17:54),masuk (09:16),12,245000
9,10,"FARHATUN MAULIDA, SE.",Farhatun Maulida-1229,NaN,STAF PEL. AKADEMIK,masuk (08:14) Keluar (17:02),masuk (08:11) Keluar (17:03),masuk (08:08) Keluar (17:16),masuk (08:16) Keluar (17:03),Tidak Masuk,Tidak Masuk,Tidak Masuk,masuk (08:02) Keluar (16:57),masuk (08:26),11,210000
